<a href="https://colab.research.google.com/github/anurag-maurya/Kaggle_MNIST_Solution_Using-_CNN/blob/master/Kaggle_MNIST_Solution_Using__CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import io
import csv


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# remove /content
df=pd.read_csv('drive/My Drive/Colab Notebooks/train.csv')
df2=pd.read_csv('drive/My Drive/test.csv')

In [0]:
df.sample(frac=1)

In [0]:
tf.reset_default_graph() 

In [0]:
train_labels=df.iloc[:,0].values
train_images=df.iloc[:,1:].values
train_images=train_images/255
test_images=df2.iloc[:,:].values
test_images=test_images/255

In [0]:
train_y=tf.one_hot(train_labels,10)

In [8]:
train_x=tf.reshape(train_images,[-1,28,28,1])
test_x=tf.reshape(test_images,[-1,28,28,1])
print(test_x.shape)

(28000, 28, 28, 1)


In [0]:
W1=tf.get_variable(name='W1',shape=[5,5,1,16],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer(seed=0))
b1=tf.get_variable(name='b1',shape=[16],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())
W2=tf.get_variable(name='W2',shape=[5,5,16,32],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer(seed=0))
b2=tf.get_variable(name='b2',shape=[32],dtype=tf.float32,initializer=tf.contrib.layers.xavier_initializer())

In [0]:
def tensor2array(p):
  with tf.name_scope("aise_hi"):
    sess=tf.Session()
    p=p.eval()
    sess.close
  return p


In [0]:
def predict_output(X):
  wx1=tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
  z1=tf.nn.relu(wx1+b1)
  a1 = tf.nn.max_pool(z1 , ksize = [1,8,8,1] , strides = [1,8,8,1] , padding = 'SAME')
  #a1=tf.nn.pool(z1,window_shape=[2,2],pooling_type='AVG',padding='SAME')
  wx2=tf.nn.conv2d(a1,W2,strides=[1,1,1,1],padding='SAME')
  z2=tf.nn.relu(wx2+b2)
  a2 = tf.nn.max_pool(z2 , ksize = [1,4,4,1] , strides = [1,4,4,1] , padding = 'SAME')
  #a2=tf.nn.pool(z2,window_shape=[2,2],pooling_type='AVG',padding='SAME')
  P2 = tf.contrib.layers.flatten(a2)
  a3=tf.contrib.layers.fully_connected(P2,num_outputs=400)
  a4=tf.contrib.layers.fully_connected(a3,num_outputs=10,activation_fn = None)
  return a4

In [0]:
batch_size=128
learning_rate=0.0003

In [0]:
sess = tf.InteractiveSession()

In [0]:
x=tf.placeholder(dtype=tf.float32,shape=[None,28,28,1])
y=tf.placeholder(dtype=tf.float32,shape=[None,10])
out_result=tf.placeholder(dtype=tf.float32,shape=[None,10])

In [0]:
y_=predict_output(x)
#y_predict=tf.argmax(y_,1)
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=y_))
train_op=tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [0]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  

In [0]:
sess.run(tf.global_variables_initializer())

In [18]:
for i in range(10):
  m=train_images.shape[0] 
  for j in range(batch_size,m,batch_size):
    batch_x=train_x[j-batch_size:j,:,:,:]
    batch_y=train_y[j-batch_size:j,:]
    sess.run(train_op,feed_dict={x:tensor2array(batch_x),y:tensor2array(batch_y)})
  print("epoch : ",i)
  print("loss=",loss.eval(feed_dict={x:tensor2array(train_x),y:tensor2array(train_y)}))
  print("train accuracy=",accuracy.eval(feed_dict={x:tensor2array(train_x),y:tensor2array(train_y)}))
  if(i==9):
    out_result=y_.eval(feed_dict={x:tensor2array(test_x)})

epoch :  0
loss= 0.4553002
train accuracy= 0.8632619
epoch :  1
loss= 0.24232073
train accuracy= 0.9257143
epoch :  2
loss= 0.17494932
train accuracy= 0.94752383
epoch :  3
loss= 0.14058441
train accuracy= 0.95709527
epoch :  4
loss= 0.1191873
train accuracy= 0.96338093
epoch :  5
loss= 0.10448679
train accuracy= 0.96828574
epoch :  6
loss= 0.09378359
train accuracy= 0.97085714
epoch :  7
loss= 0.08650248
train accuracy= 0.97333336
epoch :  8
loss= 0.08097742
train accuracy= 0.97488093
epoch :  9
loss= 0.07671486
train accuracy= 0.9759524


In [0]:
p=[]
out_result=tf.argmax(out_result,1)
p=out_result.eval()

In [0]:
with open('my_file.csv','w') as f:
  thewriter=csv.writer(f)
  thewriter.writerow(['ImageId','Label'])
  for i in range(0,out_result.shape[0]):
    thewriter.writerow([i+1,p[i]])
  

In [0]:
sess.close()
